In [1]:
!pip install --quiet ultralytics opencv-python matplotlib

from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.6 MB/s eta 0:00:00:00:0100:01
Creating new Ultral

In [2]:
import json, random, shutil
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121


In [4]:
import torch
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))



GPU available: True
GPU name: Tesla T4


In [5]:
!pip install Path

In [6]:
from pathlib import Path
IMAGES_DIR = Path("/kaggle/input/manga109-images")   # các subfolder từng bộ truyện
ANNOTATIONS_DIR = Path("/kaggle/input/manga109/jsons") 

In [7]:
# 1. Paths & logging
# =========================
yolo_data_dir = Path("./yolo_data")
train_images_dir = yolo_data_dir / "images/train"
val_images_dir   = yolo_data_dir / "images/val"
train_labels_dir = yolo_data_dir / "labels/train"
val_labels_dir   = yolo_data_dir / "labels/val"

for path in [train_images_dir, val_images_dir, train_labels_dir, val_labels_dir]:
    path.mkdir(parents=True, exist_ok=True)

IMAGES_DIR = Path("/kaggle/input/manga109-images/images")  # images dataset
ANNOTATIONS_DIR = Path("/kaggle/input/manga109-jsons/jsons")  # COCO JSONs

def log(msg):
    print(f"[INFO] {msg}")


In [8]:
# =========================
# 4. Lấy danh sách JSON & check
# =========================
json_files = list(ANNOTATIONS_DIR.rglob("*.json"))
log(f"Found {len(json_files)} JSON files")
assert len(json_files) > 0, f"No JSON files found in {ANNOTATIONS_DIR}"


[INFO] Found 109 JSON files


In [9]:
print("Folders trong images:")
for f in IMAGES_DIR.iterdir():
    print(f.name)

print("\nJSON files:")
for j in ANNOTATIONS_DIR.glob("*.json"):
    print(j.stem)


Folders trong images:
NichijouSoup
LancelotFullThrottle
ShimatteIkouze_vol01
ARMS
EvaLady
SyabondamaKieta
RinToSiteSippuNoNaka
TetsuSan
YouchienBoueigumi
Donburakokko
MagicianLoad
RisingGirl
HisokaReturns
HighschoolKimengumi_vol01
MAD_STONE
HanzaiKousyouninMinegishiEitarou
MomoyamaHaikagura
Raphael
SeisinkiVulnus
SonokiDeABC
YoumaKourin
HealingPlanet
DualJustice
BurariTessenTorimonocho
SaladDays_vol18
DollGun
GarakutayaManta
UnbalanceTokyo
KyokugenCyclone
Belmondo
OL_Lunch
MagicStarGakuin
Count3DeKimeteAgeru
GakuenNoise
TouyouKidan
BakuretsuKungFuGirl
MutekiBoukenSyakuma
EienNoWith
TasogareTsushin
MayaNoAkaiKutsu
HinagikuKenzan
BokuHaSitatakaKun
MiraiSan
YasasiiAkuma
ShimatteIkouze_vol26
KoukouNoHitotachi
PLANET7
MeteoSanStrikeDesu
Akuhamu
AisazuNihaIrarenai
ReveryEarth
AppareKappore
Arisa
TapkunNoTanteisitsu
KarappoHighschool
Jyovolley
HighschoolKimengumi_vol20
Nekodama
Joouari
YamatoNoHane
ThatIzumiko
UchuKigekiM774
EverydayOsakanaChan
AkkeraKanjinchou
MukoukizuNoChonbo
MemorySeijin


In [10]:
for jf in ANNOTATIONS_DIR.glob("*.json"):
    manga_name = jf.stem
    img_folder = IMAGES_DIR / manga_name
    print(f"JSON: {jf.name}, folder ảnh: {img_folder}, tồn tại: {img_folder.exists()}")


JSON: YouchienBoueigumi.json, folder ảnh: /kaggle/input/manga109-images/images/YouchienBoueigumi, tồn tại: True
JSON: WarewareHaOniDearu.json, folder ảnh: /kaggle/input/manga109-images/images/WarewareHaOniDearu, tồn tại: True
JSON: MisutenaideDaisy.json, folder ảnh: /kaggle/input/manga109-images/images/MisutenaideDaisy, tồn tại: True
JSON: TennenSenshiG.json, folder ảnh: /kaggle/input/manga109-images/images/TennenSenshiG, tồn tại: True
JSON: KimiHaBokuNoTaiyouDa.json, folder ảnh: /kaggle/input/manga109-images/images/KimiHaBokuNoTaiyouDa, tồn tại: True
JSON: HighschoolKimengumi_vol01.json, folder ảnh: /kaggle/input/manga109-images/images/HighschoolKimengumi_vol01, tồn tại: True
JSON: MiraiSan.json, folder ảnh: /kaggle/input/manga109-images/images/MiraiSan, tồn tại: True
JSON: LoveHina_vol01.json, folder ảnh: /kaggle/input/manga109-images/images/LoveHina_vol01, tồn tại: True
JSON: TensiNoHaneToAkumaNoShippo.json, folder ảnh: /kaggle/input/manga109-images/images/TensiNoHaneToAkumaNoShippo

In [11]:
# =========================
# ✅ 1. Import & Config
# =========================
import os
import json
import random
import shutil
import cv2
import numpy as np
import pycocotools.mask as maskUtils
from tqdm import tqdm
from pathlib import Path

# Input (gốc)
ORIGINAL_JSON_DIR = Path("/kaggle/input/manga109-jsons/jsons")
IMAGES_DIR = Path("/kaggle/input/manga109-images/images")

# Output (processed)
PROCESSED_JSON_DIR = Path("./MangaSegmentation/jsons_processed")
YOLO_DATA_DIR = Path("./yolo_data")
train_images_dir = YOLO_DATA_DIR / "images/train"
val_images_dir   = YOLO_DATA_DIR / "images/val"
train_labels_dir = YOLO_DATA_DIR / "labels/train"
val_labels_dir   = YOLO_DATA_DIR / "labels/val"

for p in [PROCESSED_JSON_DIR, train_images_dir, val_images_dir, train_labels_dir, val_labels_dir]:
    p.mkdir(parents=True, exist_ok=True)

def log(msg):
    print(f"[INFO] {msg}")


# =========================
# ✅ 2. Convert RLE → Polygon
# =========================
def preprocess_all_jsons():
    json_files = list(ORIGINAL_JSON_DIR.glob("*.json"))
    print(f"Found {len(json_files)} JSON files to process.")

    for json_file in tqdm(json_files, desc="Processing JSONs"):
        with open(json_file, 'r') as f:
            data = json.load(f)

        new_anns = []
        for ann in data["annotations"]:
            seg = ann["segmentation"]
            if isinstance(seg, dict):  # convert RLE → polygon
                mask = maskUtils.decode(seg)
                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                polys = []
                for c in contours:
                    if c.size >= 6:
                        polys.append(c.flatten().tolist())
                if not polys:
                    continue
                ann["segmentation"] = polys
            new_anns.append(ann)

        data["annotations"] = new_anns
        out_path = PROCESSED_JSON_DIR / json_file.name
        with open(out_path, "w") as f:
            json.dump(data, f)
    log(f"✅ JSONs processed and saved to {PROCESSED_JSON_DIR}")


preprocess_all_jsons()


# =========================
# ✅ 4. Split train/val
# =========================
combined = list(zip(all_image_paths, all_label_paths))
random.seed(42)
random.shuffle(combined)

split_idx = int(0.8 * len(combined))
train_set = combined[:split_idx]
val_set   = combined[split_idx:]

def copy_set(data_set, img_dest_root, label_dest_root):
    for img_path, label_path in tqdm(data_set, desc=f"Copying to {img_dest_root.name}"):
        subfolder = img_path.parent.name
        img_dest = img_dest_root / subfolder
        label_dest = label_dest_root / subfolder
        img_dest.mkdir(parents=True, exist_ok=True)
        label_dest.mkdir(parents=True, exist_ok=True)
        shutil.copy(str(img_path), str(img_dest / img_path.name))
        shutil.copy(str(label_path), str(label_dest / label_path.name))

copy_set(train_set, train_images_dir, train_labels_dir)
copy_set(val_set, val_images_dir, val_labels_dir)

log(f"Train images: {sum(len(list(p.iterdir())) for p in train_images_dir.iterdir())}")
log(f"Val images:   {sum(len(list(p.iterdir())) for p in val_images_dir.iterdir())}")

log("✅ Dataset ready for YOLO training.")


Found 109 JSON files to process.


Processing JSONs:   4%|▎         | 4/109 [01:51<48:51, 27.91s/it]


KeyboardInterrupt: 

In [ ]:
# 4. Create dataset YAML
# =========================
import yaml
dataset_yaml = yolo_data_dir / "dataset.yaml"
yaml_dict = {
    "path": str(yolo_data_dir),
    "train": "images/train",
    "val": "images/val",
    "nc": 1,
    "names": [target_class_name]
}
with open(dataset_yaml, "w") as f:
    yaml.dump(yaml_dict, f, sort_keys=False)
log(f"Dataset YAML created at: {dataset_yaml} with class: {target_class_name}")



In [ ]:
model = YOLO("yolov8n-seg.pt")  # pretrained seg model

results = model.train(
    data=str(dataset_yaml),
    epochs=50,
    imgsz=640,
    batch=8,
    device=0,  # GPU
    name="manga_bubble_seg",
    verbose=True
)


In [ ]:
# =========================
# 7. Evaluate on validation set
# =========================
val_results = model.val(
    data=str(dataset_yaml),
    batch=8,
    imgsz=640
)

# val_results chứa các metrics
print("Validation metrics:")
print(f"mAP50: {val_results.metrics.mAP_50:.4f}")
print(f"mAP50-95: {val_results.metrics.mAP_50_95:.4f}")
print(f"Precision: {val_results.metrics.precision:.4f}")
print(f"Recall: {val_results.metrics.recall:.4f}")


In [ ]:
import matplotlib.pyplot as plt

mask_mAP50 = [x['metrics/mAP50'] for x in results.history]
mask_mAP5095 = [x['metrics/mAP50-95'] for x in results.history]

plt.plot(mask_mAP50, label='mAP50')
plt.plot(mask_mAP5095, label='mAP50-95')
plt.xlabel("Epoch")
plt.ylabel("Metric")
plt.title("Training Mask mAP")
plt.legend()
plt.show()
